# Demo for launching an experiment set with tooling

Results setup and analysis : https://pad.numerique.gouv.fr/IktyaLVESTWyadnL0Q-0sw#


In [1]:
import os, sys, time, json
import dotenv
from IPython.display import HTML
import numpy as np
import pandas as pd
import requests

dotenv.load_dotenv("../.env")
sys.path.append("..")
from evalap.utils import log_and_raise_for_status

#EVALAP_API_URL = "http://localhost:8000/v1"
EVALAP_API_URL = "https://evalap.etalab.gouv.fr/v1"
EVALAP_API_KEY = os.getenv("EVALAP_API_KEY") 
ALBERT_API_URL = "https://albert.api.etalab.gouv.fr/v1"
ALBERT_API_KEY = os.getenv("ALBERT_API_KEY")
OPENAI_URL = "https://api.openai.com/v1"
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
headers = {"Authorization": f"Bearer {EVALAP_API_KEY}"}

In [ ]:
judge_name = "gpt-4.1"
judge_api_url = "https://api.openai.com/v1",
judge_api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
# Designing my experiments
# --
expset_name = "mfs_tooling_v8"
expset_readme = "Evaluating tooling capabilities.  \nembedding model: bge-multilingual-gemma2  \ncollections: chunks-v13-04-25, limit=10"
system_prompt = "Tu es un agent de l'état Français qui répond en langue française aux questions des usagers des services publiques et citoyens de manière précise, concrète et concise."
metrics = ["judge_precision", "output_length", "generation_time", "nb_tool_calls"]
common_params = {
    "dataset" : "MFS_questions_v01",
    "model": {"sampling_params" : {"temperature": 0.2}, "system_prompt": system_prompt},
    "metrics" : metrics,
    "judge_model": {
        "name": judge_name,
        "base_url": judge_api_url,
        "api_key": judge_api_key,
},
}
grid_params = {
    "model": [
        {"name": "meta-llama/Llama-3.1-8B-Instruct",                 "base_url": ALBERT_API_URL, "api_key": ALBERT_API_KEY},
        {"name": "mistralai/Mistral-Small-3.1-24B-Instruct-2503",    "base_url": ALBERT_API_URL, "api_key": ALBERT_API_KEY},
    ],
}


# Duplicate with each model params
# --
system_prompt_tool = "Tu recherche des données précises et à jour en utilisant les outils mis à disposition sur les services publiques français. Réponds de manière précise, conrète et concise."
tools = [
    #{"_tools_": ["search_albert_collections_v0"]},
    {"_tools_": ["search_albert_collections_v1"]}, # tool_choice=required not suported by albert-api atm.
    {"_tools_": ["search_albert_collections_v2"]},
]
for i in range(len(grid_params["model"])):
    for extra_params in tools:
        model = grid_params["model"][i]
        grid_params["model"].append(model.copy() | {"extra_params": extra_params} | {"system_prompt": system_prompt_tool})

# Lauching the experiment set
expset = {
    "name" : expset_name, 
    "readme": expset_readme,
    "cv": {"common_params": common_params, "grid_params": grid_params, "repeat":4}
}
response = requests.post(f'{EVALAP_API_URL}/experiment_set', json=expset, headers=headers)
resp = response.json()
if "id" in resp:
    expset_id = resp["id"]
    print(f'Created expset: {resp["name"]} ({resp["id"]})')
else:
    print(resp)

Created expset: mfs_tooling_v8 (51)
